In [1]:

import json
import os
import re
from typing import Dict, List

import numpy
from pandas import DataFrame, MultiIndex, concat, Series, array, json_normalize
PROCESS_COLUMN = ["Economy", 'Categories', 'Defense', 'General', 'Intel', 'Wreckage','Weapon','Veteran','Transport','Buffs']
STOP_FILE_NAMING = r"\w+(?=_unit)"


def read_json_units(file_path="../../data/test"):
    data = []
    for root, _, files in os.walk(file_path):
        for _file in files:
            if "json" in _file:
                with open(os.path.join(root, _file), 'r') as f:
                    unit_id = re.findall(STOP_FILE_NAMING, os.path.basename(os.path.join(root, _file)))[0]
                    data.append((unit_id,json.load(f)))
    return data
#todo make one read from file
def build_multiple_lvl_df(raw_data:Dict, index=None, select_column=None)->DataFrame:
    series = []
    def dict_to_tuple(work_dict:Dict, t_key=None):
        res = []
        for key, value in work_dict.items():
            if select_column:
                if not t_key and not key in select_column:
                    continue
            if isinstance(value, dict):
                res += [(t_key, *_) for _ in  dict_to_tuple(value, key)] if t_key else dict_to_tuple(value, key)
            elif isinstance(value,list):
                series.append(Series([value], index=[index]))
                res.append((t_key, key) if t_key else (key,))
            else:
                series.append(Series(value, index=[index]))
                res.append((t_key, key) if t_key else(key,))
        return res
    to_tuple = dict_to_tuple(raw_data)
    output_df:DataFrame = concat(series, axis=1)
    output_df.columns = MultiIndex.from_tuples(to_tuple)
    return output_df

def zip_units_dfs(units: List[DataFrame]) -> DataFrame:
    index_df = concat([_.columns.to_frame() for _ in units], axis=0).drop_duplicates()
    global_columns = MultiIndex.from_frame(index_df)
    common_df = concat([_.reindex(columns=global_columns) for _ in units], axis=0)
    common_df.columns = global_columns
    return common_df

t = build_multiple_lvl_df({"a":"v","t":[6,7,8],"b":{"a":"v","y":"x", "p":[1,2,3], "c":{"a":"v"}}}, 1)
df = t["b"]
df.columns = df.columns.get_level_values(0)
df
# t.loc[:,[("a",numpy.nan, numpy.nan), ("t","value", numpy.nan)]]

,a,y,p,c
1,v,x,"[1, 2, 3]",v


In [ ]:
#todo asked on stackoverflow
columns = MultiIndex.from_tuples(
    # [("t0",),("a","t1"),("a","t2"),("a", "b", "t3"),("a", "b", "c", "t4")],
    [("a","t0"),("b","t1"),("b","t2"),("c", "t1"),("c", "t2")],
                     # names=["lvl1","lvl2","lvl3","lvl4"])
                     names=["lvl1","lvl2"])
df = concat([DataFrame({"df1":numpy.random.randint(0,2,6)}),
             DataFrame({"df2_1":numpy.random.randint(2,4,6),
                        "df2_2":numpy.random.randint(4,6,6)}),
             DataFrame({"df4":numpy.random.randint(6,7,6)}),
             DataFrame({"df5":numpy.random.randint(7,8,6)}),], axis=1)
df.columns = columns
# df.plot.line()
df.loc[:,[("a","t0"),("c","t2")]]

# df
#
# idx = MultiIndex.from_tuples([(1, 'one', "t"), (1, 'two', "t"),
#                                     (2, 'one', "t"), (2, 'two', "t"),
#                                     (3, 'one', "t"), (3, 'two', "t")],
#                                     names=['foo', 'bar', 'new'])
# print(idx.set_levels([["a","b","c"], [1,2,3], ["d","f","g"]],level=[1,2,3]))
# df = DataFrame(data, columns=PROCESS_COLUMN, index=list_unit_id)

In [ ]:
dfs = []
for _id,content in read_json_units():
    dfs.append(build_multiple_lvl_df(content[0], _id, PROCESS_COLUMN))
t = concat(dfs, axis=1)
# t.info()
df = t["Economy"]
df.columns = df.columns.get_level_values(0)
df

In [55]:
all_units:List[DataFrame] = []
for _id, content in read_json_units("/Users/testolog/faf_repo/json")[::15]:
    for data in content:
        lvl_df = build_multiple_lvl_df(data, _id, PROCESS_COLUMN)
        all_units.append(lvl_df)


# if not os.path.exists("/Users/testolog/faf_repo/all_units.json"):
#     all_df = zip_units_dfs(all_units)
#     all_df.to_json("/Users/testolog/faf_repo/all_units.json")
# else:
#     all_df = read_json_units("/Users/testolog/faf_repo/all_units.json")
all_df:DataFrame = zip_units_dfs(all_units)

In [156]:
categories:DataFrame = all_df.loc[:, ["General"]].copy()
# categories.columns = categories.columns.get_level_values(1)
categories["id_unit"] = categories.index
categories.reset_index(inplace=True, drop=True)
# categories.set_index(categories.loc[:,("id")])
# categories.set_index([("General","Category"),("General","FactionName"), "id_unit"], inplace=True)
# categories.sort_index(level=[0], inplace=True)
categories.columns.set_names()
# categories.columns = MultiIndex.from_frame(categories.columns.to_frame().drop_duplicates())
categories.loc[::3,::10]

0        General                                                       \
1       Category UpgradesTo     CommandCaps                             
2            NaN        NaN RULEUCC_Reclaim RULEUCC_SiloBuildTactical   
3            NaN        NaN             NaN                       NaN   
4            NaN        NaN             NaN                       NaN   
5            NaN        NaN             NaN                       NaN   
0        Factory    uab0302           false                       NaN   
3        Defense        NaN           false                       NaN   
6        Defense        NaN           false                      true   
9        Fighter        NaN           false                       NaN   
12       Defense        NaN           false                       NaN   
15       Defense        NaN           false                       NaN   
18  Construction        NaN            true                       NaN   
21       Gunship        NaN           false                       NaN   
24  IndirectFire        NaN           false                       NaN   
27       Factory        NaN           false                       NaN   
30       Defense        NaN           false                       NaN   
33  Intelligence    ueb3202           false                       NaN   

0                         
1                         
2  RULEUCC_SiloBuildNuke  
3                    NaN  
4                    NaN  
5                    NaN  
0                    NaN  
3                    NaN  
6                    NaN  
9                    NaN  
12                   NaN  
15                   NaN  
18                   NaN  
21                   NaN  
24                   NaN  
27                   NaN  
30                   NaN  
33                   NaN

In [3]:
x = {"a":"v","t":[6,7,8],"b":{"a":"v","y":"x", "p":[1,2,3], "c":{"a":"v"}}}

df_x = json_normalize(x)
df_x

,a,t,b.a,b.y,b.p,b.c.a
0,v,"[6, 7, 8]",v,x,"[1, 2, 3]",v


In [12]:
a=[1,2,3,4,5]
b =(_ for _ in a if _ > 29)
_next = next(b, 0)
_next
b1, b2 = (1,2)
print(b1,b2)


1 2


In [14]:

import itertools as it
list(it.product(["a","b"], [1,2]))

[('a', 1), ('a', 2), ('b', 1), ('b', 2)]

In [15]:
list(it.product(["a","b", "c"], [1,2]))


[('a', 1), ('a', 2), ('b', 1), ('b', 2), ('c', 1), ('c', 2)]